In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [16]:


# BasicBlock as defined in the ResNet paper
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

# ResNet model as defined in the ResNet paper
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):  # Changed num_classes to 10 for CIFAR-10
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)  # Adjusted for CIFAR-10
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = torch.nn.functional.avg_pool2d(out, 4)  # Correct function call
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

# Load the CIFAR-10 dataset
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Initialize the ResNet-18 model, loss function, and optimizer
device = torch.device('cpu')
model = ResNet18().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Training function
def train(epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx % 100 == 0:
            print(f'Epoch: {epoch} | Batch: {batch_idx+1}/{len(trainloader)} | Loss: {train_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f}%')

# Testing function
def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    print(f'Test | Loss: {test_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f}%')

# Train and test the model for 5 epochs
for epoch in range(1, 6):
    train(epoch)
    test(epoch)


Files already downloaded and verified
Files already downloaded and verified
Epoch: 1 | Batch: 1/391 | Loss: 2.333 | Acc: 8.594%
Epoch: 1 | Batch: 101/391 | Loss: 2.390 | Acc: 21.303%
Epoch: 1 | Batch: 201/391 | Loss: 2.086 | Acc: 26.873%
Epoch: 1 | Batch: 301/391 | Loss: 1.937 | Acc: 30.749%
Test | Loss: 1.494 | Acc: 45.200%
Epoch: 2 | Batch: 1/391 | Loss: 1.512 | Acc: 44.531%
Epoch: 2 | Batch: 101/391 | Loss: 1.472 | Acc: 46.279%
Epoch: 2 | Batch: 201/391 | Loss: 1.430 | Acc: 47.979%
Epoch: 2 | Batch: 301/391 | Loss: 1.392 | Acc: 49.136%
Test | Loss: 1.228 | Acc: 55.460%
Epoch: 3 | Batch: 1/391 | Loss: 1.117 | Acc: 60.938%
Epoch: 3 | Batch: 101/391 | Loss: 1.152 | Acc: 58.779%
Epoch: 3 | Batch: 201/391 | Loss: 1.116 | Acc: 59.907%
Epoch: 3 | Batch: 301/391 | Loss: 1.089 | Acc: 61.015%
Test | Loss: 0.977 | Acc: 65.410%
Epoch: 4 | Batch: 1/391 | Loss: 0.827 | Acc: 73.438%
Epoch: 4 | Batch: 101/391 | Loss: 0.925 | Acc: 67.489%
Epoch: 4 | Batch: 201/391 | Loss: 0.914 | Acc: 67.673%
Epoch: